###  Desafio - Etapa1 🚀
---

Nessa etapa do desafio, será realizada a leitura de um arquivo oriundo do portal de dados públicos do Governo Brasileiro, o csv "chegadas_2024.csv" em que reúne dados relativos às estimativas do fluxo de chegadas de turistas internacionais (incluindo turistas estrangeiros e brasileiros que residem no exterior) ao Brasil, desagregadas por país de residência permanente, por mês e via de acesso (aérea, terrestre, marítima ou fluvial). Assim, será feito um tratamento de dados inicial para posteriormente analisar as informações contidas no mesmo. Seguirei uma sequência de ações nas quais estarei detalhando um pouco mais sobre os códigos que usei para execução dos passos.

Análise inicial
__________________________________________________________________________________________________
Inicialmente importei a biblioteca pandas para análise e tratamento dos dados, durante a análise inicial do conjunto de dados chegadas_2024.csv, ao tentar carregá-lo com pd.read_csv(), ocorreu um erro relacionado à codificação do arquivo. Para identificar o encoding correto, utilizei a biblioteca chardet, que indicou a codificação 'ISO-8859-1'. Após ajustar o parâmetro de codificação, ainda enfrentei um erro de tokenização devido ao separador de colunas, que não era a vírgula padrão. Especificando o separador como ';', foi possível realizar a leitura correta do arquivo e visualizar suas primeiras entradas e informações gerais.

Além disso, importei as bibliotecas boto3 e botocore.exceptions para realizar operações com o serviço Amazon S3, como a criação do bucket e o envio de arquivos para a nuvem. Também utilizei dotenv para carregar variáveis de ambiente armazenadas no arquivo .env, garantindo segurança e flexibilidade na autenticação, a biblioteca os também foi utilizada para acessar essas variáveis carregadas.

In [8]:
import pandas as pd
import chardet
import boto3
import os

from botocore.exceptions import ClientError
from dotenv import load_dotenv

In [2]:
with open('chegadas_2024.csv', 'rb') as f:
    result = chardet.detect(f.read(10000))
    print(result)

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [3]:
df = pd.read_csv('chegadas_2024.csv', encoding='ISO-8859-1', sep=';')


In [16]:
df.head()

,Continente,cod continente,País,cod pais,UF,cod uf,Via,cod via,ano,Mês,cod mes,Chegadas
0,Europa,6,Alemanha,57,Outras Unidades da Federação,99,Aérea,1,2024,Janeiro,1,10
1,América do Sul,4,Argentina,26,Outras Unidades da Federação,99,Aérea,1,2024,Janeiro,1,1197
2,Europa,6,Bélgica,59,Outras Unidades da Federação,99,Aérea,1,2024,Janeiro,1,3
3,América do Norte,3,Canadá,23,Outras Unidades da Federação,99,Aérea,1,2024,Janeiro,1,2
4,América do Sul,4,Chile,28,Outras Unidades da Federação,99,Aérea,1,2024,Janeiro,1,1


- Conhecendo as dimensões do DataFrame:

In [17]:
df.shape

(22856, 12)

- Conhecendo os últimos registros do DataFrame:

In [18]:
df.tail()

,Continente,cod continente,País,cod pais,UF,cod uf,Via,cod via,ano,Mês,cod mes,Chegadas
22851,Europa,6,Suíça,86,Rio Grande do Sul,21,Fluvial,4,2024,Dezembro,12,3
22852,América do Sul,4,Uruguai,36,Rio Grande do Sul,21,Fluvial,4,2024,Dezembro,12,6
22853,América do Sul,4,Argentina,26,Santa Catarina,24,Fluvial,4,2024,Dezembro,12,120
22854,América do Sul,4,Chile,28,Santa Catarina,24,Fluvial,4,2024,Dezembro,12,3
22855,América do Sul,4,Uruguai,36,Santa Catarina,24,Fluvial,4,2024,Dezembro,12,1


- Informações da base (tipos de dados, linhas não nulas, número de entradas):

In [19]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22856 entries, 0 to 22855
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Continente      22856 non-null  object
 1   cod continente  22856 non-null  int64 
 2   País            22856 non-null  object
 3   cod pais        22856 non-null  int64 
 4   UF              22856 non-null  object
 5   cod uf          22856 non-null  int64 
 6   Via             22856 non-null  object
 7   cod via         22856 non-null  int64 
 8   ano             22856 non-null  int64 
 9   Mês             22856 non-null  object
 10  cod mes         22856 non-null  int64 
 11  Chegadas        22856 non-null  int64 
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


- Verificando quantos valores únicos existem em cada coluna:

In [20]:
df.nunique()

Continente           8
cod continente       8
País                93
cod pais            93
UF                  18
cod uf              18
Via                  4
cod via              4
ano                  1
Mês                 12
cod mes             12
Chegadas          1702
dtype: int64

- Visualizando os valores vazios:

In [21]:
df.isnull().sum()

Continente        0
cod continente    0
País              0
cod pais          0
UF                0
cod uf            0
Via               0
cod via           0
ano               0
Mês               0
cod mes           0
Chegadas          0
dtype: int64

###  Questionamentos 
---

Por meio da análise inicial do arquivo, foi possível identificar as principais informações para compreender o comportamento da chegada de turistas ao Brasil ao longo do ano. Com base nisso, selecionei três questionamentos principais que considero estratégicos para gerar insights sobre o fluxo turístico no país:

1. Classificação dos países por faixas de volume de turistas: a partir dessa análise é possível identificar quais países enviaram mais turistas ao Brasil e quais apresentam menor representatividade;
2. Principais vias de entrada de turistas no Brasil: ajuda a entender por quais principais vias os turistas mais chegam ao país;
3. Meses com maior número de chegadas de turistas via aérea em 2024: mostra a sazonalidade do turismo ao longo do ano por meio das vias aéreas.


###  Criação do bucket e upload do arquivo
---

Após a análise exploratória dos dados do arquivo chegadas_2024.csv, realizei o envio do mesmo para a nuvem utilizando o serviço Amazon S3, garantindo que os dados estejam acessíveis para as próximas etapas do projeto. Inicialmente, carreguei as variáveis de ambiente armazenadas no arquivo .env com a função load_dotenv(), assegurando a segurança das credenciais de autenticação (chave de acesso, segredo e token da sessão). Essas credenciais foram usadas para configurar o cliente do S3 via biblioteca boto3. Defini o nome do bucket como chegadas-turistas-2024-ana e, em seguida, executei sua criação por meio de um bloco try-except, tratando o cenário em que o bucket já exista e seja de minha propriedade. Vale ressaltar que, como a região utilizada foi a us-east-1, a criação do bucket foi realizada sem o parâmetro explícito de região, em conformidade com as regras da AWS para essa localidade. Com o bucket validado, o próximo passo foi o upload do arquivo chegadas_2024.csv, também encapsulado em um bloco de tratamento de exceções para lidar com possíveis falhas durante o envio.

In [ ]:
load_dotenv()
env = os.environ

bucket_name = "chegadas-turistas-2024-ana"
arquivo_local = "chegadas_2024.csv"
arquivo_s3 = "chegadas_2024.csv"

region = env.get("AWS_REGION")

s3 = boto3.client(
    's3',
    aws_access_key_id=env.get("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=env.get("AWS_SECRET_ACCESS_KEY"),
    aws_session_token=env.get("AWS_SESSION_TOKEN"),
    region_name=region
)

try:
    s3.create_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' criado com sucesso!")
except ClientError as e:
    if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
        print(f"Bucket '{bucket_name}' já existe e pertence a você.")
    else:
        raise

try:
    s3.upload_file(arquivo_local, bucket_name, arquivo_s3)
    print(f"Arquivo '{arquivo_local}' enviado com sucesso para o bucket '{bucket_name}'.")
except Exception as e:
    print("Erro ao fazer upload:", e)

Bucket 'chegadas-turistas-2024-ana' criado com sucesso!
Arquivo 'chegadas_2024.csv' enviado com sucesso para o bucket 'chegadas-turistas-2024-ana'.
